In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from typing import TypedDict
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent, AgentState
from langchain.tools import tool 
import os
from langchain.agents.middleware import wrap_tool_call, dynamic_prompt, ModelRequest
from langchain_core.messages import ToolMessage
from pydantic import BaseModel, Field
from typing import Optional
import pandas as pd

e:\pet\food\AI-cook\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

kk = pd.read_excel("kk.xlsx")

def parse_range(value):
    """Преобразует строку с диапазоном в среднее число"""
    if pd.isna(value):
        return None
    try:
        # Заменяем запятые на точки и разделяем по дефису
        parts = str(value).replace(',', '.').split('-')
        # Убираем пробелы и преобразуем в числа
        numbers = [float(part.strip()) for part in parts]
        # Возвращаем среднее значение
        return sum(numbers) / len(numbers)
    except:
        return None

# Применяем ко всем столбцам с диапазонами
kk['squirrels_avg'] = kk['squirrels'].apply(parse_range)
kk['fats_avg'] = kk['fats'].apply(parse_range)
kk['carbohydrates_avg'] = kk['carbohydrates'].apply(parse_range)
kk['callories_avg'] = kk['callories'].apply(parse_range)
kk

,Product,squirrels,fats,carbohydrates,callories,squirrels_avg,fats_avg,carbohydrates_avg,callories_avg
0,Хлеб пшеничный из муки высшего сорта,"7,5-7,6","0,8-2,9","49,2-51,4",235-262,7.55,1.85,50.3,248.5
1,Хлеб пшеничный из муки 1 сорта,7.6,0.9,49.7,226,7.60,0.90,49.7,226.0
2,Хлеб из ржано-пшеничной муки,6.8,1.2,46.4,215,6.80,1.20,46.4,215.0
3,Хлеб ржаной,4.7,0.7,49.8,214,4.70,0.70,49.8,214.0
4,"Хлеб ""Бородинский""",6.9,1.3,40.9,208,6.90,1.30,40.9,208.0
...,...,...,...,...,...,...,...,...,...
631,Саке,0.5,-,5,134,0.50,NaN,5.0,134.0
632,Самогон,0.1,0.1,0.4,235,0.10,0.10,0.4,235.0
633,Спирт этиловый 96%,4,-,3.8,710,4.00,NaN,3.8,710.0
634,Текила,1.4,0.3,24,231,1.40,0.30,24.0,231.0


In [3]:
kk[kk["Product"] == "Спирулина сушеная"]

,Product,squirrels,fats,carbohydrates,callories,squirrels_avg,fats_avg,carbohydrates_avg,callories_avg
422,Спирулина сушеная,57.5,7.72,23.4,290,57.5,7.72,23.4,290.0


In [29]:
class ProductSearch(BaseModel):
    """Ввод данных для запросов о еде"""

    name: Optional[str] = Field(description="Название продукта", default=None)
    squirrels: Optional[str] = Field(description="Белки", default=None)
    fats: Optional[str] = Field(description="Жиры", default=None)
    carbohydrates: Optional[str] = Field(description="Углеводы", default=None)
    callories: Optional[str] = Field(description="Энергия", default=None)
    squirrels_avg: Optional[str] = Field(description="Белки среднее значение", default=None)
    fats_avg: Optional[str] = Field(description="Жиры среднее значение", default=None)
    carbohydrates_avg: Optional[str] = Field(description="Углеводы среднее значение", default=None)
    callories_avg: Optional[str] = Field(description="Энергия среднее значение", default=None)
    
    # Добавляем параметры для диапазонного поиска
    min_squirrels: Optional[int] = Field(description="Минимальное количество белков", default=None)
    max_squirrels: Optional[int] = Field(description="Максимальное количество белков", default=None)
    min_fats: Optional[int] = Field(description="Минимальное количество жиров", default=None)
    max_fats: Optional[int] = Field(description="Максимальное количество жиров", default=None)
    min_carbohydrates: Optional[int] = Field(description="Минимальное количество углеводов", default=None)
    max_carbohydrates: Optional[int] = Field(description="Максимальное количество углеводов", default=None)
    min_callories: Optional[int] = Field(description="Минимальная энергетическая ценность", default=None)
    max_callories: Optional[int] = Field(description="Максимальная энергетическая ценность", default=None)
    number_products: Optional[int] = Field(description="Количество продуктов", default=None)
    
    sort_order: Optional[str] = Field(
        description="Порядок сортировки (наибольшие белки, наименьшие белки, наибольшие жиры, наименьшие жиры, наибольшие углеводы, наименьшие углеводы,наибольшие каллории, наименьшие каллории)",
        default=None,
    )

In [30]:
@tool(args_schema=ProductSearch)
def search_food(
    name: Optional[str] = None,
    squirrels: Optional[str] = None,
    fats: Optional[str] = None,
    carbohydrates: Optional[str] = None,
    callories: Optional[str] = None,
    squirrels_avg: Optional[str] = None,
    fats_avg: Optional[str] = None,
    carbohydrates_avg: Optional[str] = None,
    callories_avg: Optional[str] = None,
    min_squirrels: Optional[int] = None,
    max_squirrels: Optional[int] = None,
    min_fats: Optional[int] = None,
    max_fats: Optional[int] = None,
    min_carbohydrates: Optional[int] = None,
    max_carbohydrates: Optional[int] = None,
    min_callories: Optional[int] = None,
    max_callories: Optional[int] = None,
    sort_order: Optional[str] = None,
    number_products: Optional[int] = None
) -> str:
    """Поиск продуктов питания по различным критериям питания"""
    
    x = kk.copy()
    
    # Apply filters
    if name:  
        x = x[x["Product"].apply(lambda x: name.lower() in str(x).lower())]
    if min_callories: 
        x = x[x["callories_avg"] >= min_callories]
    if max_callories: 
        x = x[x["callories_avg"] <= max_callories]
    if min_carbohydrates: 
        x = x[x["carbohydrates_avg"] >= min_carbohydrates]
    if max_carbohydrates: 
        x = x[x["carbohydrates_avg"] <= max_carbohydrates]
    if min_fats: 
        x = x[x["fats_avg"] >= min_fats]
    if max_fats: 
        x = x[x["fats_avg"] <= max_fats]
    if min_squirrels: 
        x = x[x["squirrels_avg"] >= min_squirrels]
    if max_squirrels: 
        x = x[x["squirrels_avg"] <= max_squirrels]
    if sort_order:
        if sort_order == "наибольшие белки":x = x.sort_values('squirrels_avg', ascending=False)
        if sort_order == "наименьшие белки":x = x.sort_values('squirrels_avg', ascending=True)
        if sort_order == "наибольшие жиры":x = x.sort_values('fats_avg', ascending=False)
        if sort_order == "наименьшие жиры":x = x.sort_values('fats_avg', ascending=True)
        if sort_order == "наибольшие углеводы":x = x.sort_values('carbohydrates_avg', ascending=False)
        if sort_order == "наименьшие углеводы":x = x.sort_values('carbohydrates_avg', ascending=True)
        if sort_order == "наибольшие каллории":x = x.sort_values('callories_avg', ascending=False)
        if sort_order == "наименьшие каллории":x = x.sort_values('callories_avg', ascending=True)
    if number_products: x = x.head(number_products)
    # Формируем результат
    result = "Результаты поиска продуктов:\n"
    
    for i, (idx, food) in enumerate(x.iterrows(), 1):
        result += f"{i}. {food['Product']} - Б: {food.get('squirrels_avg', 'N/A')}г, Ж: {food.get('fats_avg', 'N/A')}г, У: {food.get('carbohydrates_avg', 'N/A')}г, Кал: {food.get('callories_avg', 'N/A')}\n"
    
    result += f"\nНайдено продуктов: {len(x)}"
    
    return result

In [14]:
carts = {}

class AddToCart(BaseModel):
    """Ввод данных для добавления продукта в корзину"""
    name: str = Field(description="Точное название еды, чтобы положить в корзину", default=None)
    count: float = Field(description="Количество, которое нужно положить в корзину. Измеряется в граммах",default=100.0)

@tool(args_schema=AddToCart)
def add_product_to_cart(
    name: Optional[str] = None,
    count: Optional[float] = 100.0
    ):
    """Добавление продуктов в коризну carts"""
    
    if name not in kk['Product'].values:
        return f"Продукт '{name}' не найден. Пожалуйста, проверьте название."
    
    # Добавляем в корзину
    carts[name] = carts.get(name, 0) + count
    
    return f'{count} грамм "{name}" добавлено в корзину. Теперь в корзине: {carts[name]} грамм'


In [15]:
@tool(description="Показывает содержимое корзины")
def show_product_in_cart():
    """Показать продукты в корзине carts"""
    
    return carts

In [16]:
base_prompt = """Ты полезный ассистент по питанию. Ты получил результаты поиска продуктов. 
            Сформируй красивый, читаемый ответ для пользователя на русском языке.

            Если нужен список продуктов вот структурированный ответ:
            1. Краткое введение
            2. Группируй продукты по категориям (мясо, рыба, молочные и т.д.)
            3. Для каждого продукта укажи ключевые показатели
            4. Дай рекомендации

            Используй эмодзи и понятное форматирование."""

In [31]:
llm = ChatOpenAI(
    model="Qwen/Qwen3-235B-A22B-Instruct-2507",
    openai_api_key=os.environ.get("GIGA"),
    openai_api_base="https://foundation-models.api.cloud.ru/v1",
    max_tokens=1000
)


@tool
def get_weather(location: str) -> str:
    """Получить информацию о погоде для местоположения."""
    return f"Погода в {location}: Солнечно, 22°C"

@wrap_tool_call
def handle_tool_errors(request, handler):
    """Обработка ошибок выполнения инструментов с пользовательскими сообщениями."""
    try:
        return handler(request)
    except Exception as e:
        # Возвращаем пользовательское сообщение об ошибке модели
        return ToolMessage(
            content=f"Ошибка инструмента: Пожалуйста, проверьте ваш ввод и попробуйте снова. ({str(e)})",
            tool_call_id=request.tool_call["id"]
        )

class Context(TypedDict):
    user_role: str

@dynamic_prompt
def user_role_prompt(request: ModelRequest) -> str:
    """Генерация системного промпта на основе роли пользователя."""
    user_role = request.runtime.context.get("user_role", "user")
    

    if user_role == "expert":
        return f"{base_prompt} Предоставляйте подробные технические ответы."
    elif user_role == "beginner":
        return f"{base_prompt} Объясняйте концепции просто и избегайте профессионального жаргона."

    return base_prompt

agent = create_agent(
    llm,
    tools=[get_weather,search_food,add_product_to_cart,show_product_in_cart],
    middleware=[user_role_prompt,handle_tool_errors],
    context_schema=Context
)

# Системный промпт будет устанавливаться динамически на основе контекста


In [32]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Посоветуй 2 продукта, где больше 25 белков"}]},
    context={"user_role": "expert"}
)

In [33]:
result["messages"]

[HumanMessage(content='Посоветуй 2 продукта, где больше 25 белков', additional_kwargs={}, response_metadata={}, id='b1fd9ba1-967c-44ef-80d4-b26efa175417'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 1350, 'total_tokens': 1381, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 208}}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen3-235B-A22B-Instruct-2507', 'system_fingerprint': None, 'id': 'chatcmpl-66e2a297-bbb5-4ac7-8c4e-57cca1705f2b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--e224fa6c-aec7-41ed-ba28-de304c2295c9-0', tool_calls=[{'name': 'search_food', 'args': {'min_squirrels': 25, 'number_products': 2}, 'id': 'chatcmpl-tool-f5cb000b43b74875b0482789201e2b95', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1350, 'output_tokens': 31, 'total_tokens': 1381, 'input_token_details': {'cache_read': 208}, 'output_token

In [34]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Добавь в корзину Соевые бобы, 500 грамм"}]},
    context={"user_role": "expert"}
)

In [35]:
result

{'messages': [HumanMessage(content='Добавь в корзину Соевые бобы, 500 грамм', additional_kwargs={}, response_metadata={}, id='7480b395-82a2-45b2-805c-cccef4cf9232'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 1352, 'total_tokens': 1386, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 1328}}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen3-235B-A22B-Instruct-2507', 'system_fingerprint': None, 'id': 'chatcmpl-6de56dbd-3d1d-43ec-b2e4-111549fd985e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--4161c04f-4e86-4f5a-88cc-487e827f3631-0', tool_calls=[{'name': 'add_product_to_cart', 'args': {'name': 'Соевые бобы', 'count': 500}, 'id': 'chatcmpl-tool-8f0036cc58f34afd999f2a47d1e51d55', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1352, 'output_tokens': 34, 'total_tokens': 1386, 'input_token_details': {'cache_read': 1328}

In [36]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Что в моей корзине?"}]},
    context={"user_role": "expert"}
)

In [37]:
result

{'messages': [HumanMessage(content='Что в моей корзине?', additional_kwargs={}, response_metadata={}, id='20679a30-4126-4d49-b215-54d6913dcd93'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 1343, 'total_tokens': 1361, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 1328}}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen3-235B-A22B-Instruct-2507', 'system_fingerprint': None, 'id': 'chatcmpl-c0b747a4-489b-4e8e-b74e-659baa349c24', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--7265dad8-6a4d-4a76-b607-65aa743a7b01-0', tool_calls=[{'name': 'show_product_in_cart', 'args': {}, 'id': 'chatcmpl-tool-7a33f83ecea4457d9ef33df8335dcc62', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1343, 'output_tokens': 18, 'total_tokens': 1361, 'input_token_details': {'cache_read': 1328}, 'output_token_details': {}}),
  ToolMessage(content=